In [1]:
%load_ext autoreload
%autoreload 2

import os
import pickle as pkl
import subprocess
from pathlib import Path

import napari
from magicgui import magicgui

from plex_pipe.core_definition.viewer_utils import (
    display_saved_rois,
    redo_bbox_layer,
    redo_cores_layer,
    save_rois_from_viewer,
)
from plex_pipe.utils.im_utils import get_org_im_shape, prepare_rgb_image
from plex_pipe.core_definition.roi_utils import (
    get_refined_rectangles,
    get_visual_rectangles,
    prepare_poly_df_for_saving,
    xywh_to_corners,
)
from plex_pipe.utils.file_utils import (
    change_to_wsl_path
)
from plex_pipe.utils.config_loaders import (
    load_analysis_settings,
    load_workstation_config
)

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


## Read in config

In [2]:
# load pipeline configuration
config = load_workstation_config(r'R:\Wayne\BLCA\pipeline_settings.yaml')
config

{'model_path': '/mnt/d/data_analysis/2024_bladder/sam2',
 'sam_env': '/home/kasia/miniforge3/envs/sam2-env/bin/python'}

In [3]:
# load analysis configuration
settings_path = r"C:\BLCA\BLCA-1A_Analysis\analysis_BLCA1A.yaml"

settings = load_analysis_settings(settings_path)
settings

2026-01-06 15:42:04.656 | WARNING  | plex_pipe.processors.base:_log_missing_optional_params:51 - InstansegSegmenter.Params: Parameter overlap not provided. Using default value: 80.


AnalysisConfig(general=GeneralSettings(image_dir='R:/CellDive/BLCA-1A/BLCA-1A_Final', analysis_name='BLCA-1A_Analysis', local_analysis_dir='C:/BLCA', remote_analysis_dir='/ix1/kkedziora/BLCA', log_dir=None), core_detection=CoreDetectionSettings(detection_image='BLCA-1A_1.0.4_R000_DAPI__FINAL_F.ome.tif', core_info_file_path=None, im_level=6, min_area=2000, max_area=20000, min_iou=0.8, min_st=0.9, min_int=15, frame=4), core_cutting=CoreCuttingSettings(cores_dir_tif=None, cores_dir_output=None, include_channels=None, exclude_channels=['008_ECad'], use_markers=None, ignore_markers=['Antibody1', 'TNFa', 'Snail1', 'SKP2', 'ProgRc', 'Plk1', 'PH3', 'PDL1', 'p65', 'p130', 'p-p130', 'p-Cdc6', 'LAG3', 'IL-8', 'HER2', 'ERa', 'EpCAM', 'E2F1', 'cycD3', 'cycB2', 'CDC25C', 'CD86', 'CD73', 'CD69', 'CD62L', 'CD56', 'CD4', 'CD25', 'CD19', 'CD27', 'CCR7', 'cCASP3'], margin=0, mask_value=0, transfer_cleanup_enabled=True, core_cleanup_enabled=True), additional_elements=[NormalizeStep(category='image_transfo

## Read in data

In [4]:
# read in image data

# simplify variable names
im_level = settings.core_detection.im_level
save_path = settings.core_info_file_path

# define the image path
image_path = Path(settings.general.image_dir) / settings.core_detection.detection_image

# get the original image shape
org_im_shape = get_org_im_shape(image_path)

# prepare the image for display
im_rgb = prepare_rgb_image(image_path, req_level = im_level)
im = im_rgb[:, :, 0]
im_rgb.shape

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\zarr\creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


(992, 1498, 3)

## Create a viewer with options

In [5]:
viewer = napari.Viewer()
viewer.add_image(im)

# add a red rectangle to frame the image
frame_rect = xywh_to_corners([0,0,im.shape[1],im.shape[0]], frame = 0)
viewer.add_shapes(frame_rect, edge_color='white', face_color = 'transparent', shape_type='rectangle', edge_width=2, name = 'frame')

# add a layer for the saved rois
display_saved_rois(viewer, IM_LEVEL = im_level, save_path = save_path)


#########################################################################################
# add diplay saved button
@magicgui(auto_call=False, call_button="Display Saved Cores", result_widget=False)
def display_saved_rois_button(viewer: napari.Viewer):
    display_saved_rois(viewer, IM_LEVEL = im_level, save_path = save_path)

# Add widget to viewer
viewer.window.add_dock_widget(display_saved_rois_button, area='left')

#########################################################################################
# add a save button
@magicgui(auto_call=False, call_button="Save Cores")
def save_button(viewer: napari.Viewer):
    save_rois_from_viewer(viewer, org_im_shape = org_im_shape, req_level = im_level, save_path = save_path)

# Add widget to viewer
viewer.window.add_dock_widget(save_button, area='left')

## Get suggestions for core detection from the SAM2 model

Execute the cell below to get suggestions from SAM model about the cores position. 

Sometimes there is no immediate feedback of how SAM2 progresses. In this situation it's expected to wait 1-2 min for the outcome.

In [51]:
# Linux path to the image
im_wsl = change_to_wsl_path(str(image_path))

# temporary output path for masks
out_win = os.path.join(os.getcwd(), 'masks.pkl')
out_wsl = change_to_wsl_path(out_win)

# Combine the command
command = [
    "wsl",
    config['sam_env'],
    "-m", "plex_pipe.core_definition.suggest_cores",
    im_wsl,
    str(im_level),
    config['model_path'],
    out_wsl
]

# Execute the script
process = subprocess.Popen(
    command,
    stdout=subprocess.PIPE,  # Pipe the standard output
    stderr=subprocess.PIPE,  # Pipe the standard error
    text=True                # Capture output as text
)

# Display output in real time
while True:
    output = process.stdout.readline()
    if output == "" and process.poll() is not None:
        break
    if output:
        print(output, end="")  # Print each line as it becomes available

# Print any remaining errors
errors = process.stderr.read()
if errors:
    print(f"ERROR: {errors}")

# load the masks
masks = pkl.load(open(out_win, 'rb'))
print(f'Initial number of masks: {len(masks)}')

# remove the masks file
os.remove(out_win)

# refine masks
rect_list = get_refined_rectangles(masks, im = im_rgb[:,:,0],
                                   frame = settings.core_detection.frame,
                                   min_area = settings.core_detection.min_area,
                                   max_area = settings.core_detection.max_area,
                                   min_iou = settings.core_detection.min_iou,
                                   min_stability = settings.core_detection.min_st,
                                   min_int = settings.core_detection.min_int
                                   )
print('Number of refined masks:', len(rect_list))

# prepare df
df = prepare_poly_df_for_saving(rect_list, poly_types = ['rectangle']*len(rect_list), org_im_shape = org_im_shape, req_level = im_level)
rect_list = get_visual_rectangles(df, im_level)
poly_list = [(x/(2**im_level)).astype('int') for x in df.polygon_vertices.to_list()]

# add to napari
redo_cores_layer(viewer,rect_list,shape_type = df.poly_type.to_list())
redo_bbox_layer(viewer,rect_list,df['core_name'].tolist())

Input image: /mnt/r/CellDive/BLCA-1A/BLCA-1A_Final/BLCA-1A_1.0.4_R000_DAPI__FINAL_F.ome.tif
Results will be saved to: /mnt/d/plex_pipe/notebooks/masks.pkl

Preparing RGB image for segmentation...
Segmenting image. It should take around 1 min. Started at 2025-11-21 12:53:24...
Saving masks...
ERROR: /home/kasia/miniforge3/envs/sam2-env/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/kasia/miniforge3/envs/sam2-env/lib/python3.11/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setup

D:\plex_pipe\src\plex_pipe\core_definition\roi_utils.py:41: RuntimeWarning: Mean of empty slice.
  mean_int = im[rect[0, 0] : rect[2, 0], rect[0, 1] : rect[1, 1]].mean()
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
